In [38]:
import numpy as np
import pandas as pd
from nltk.metrics import edit_distance

In [2]:
# reading csv of all candidates declared as rep or dem
candidate_df = pd.read_csv("candidatelist.csv", header=2)

In [3]:
candidate_df.head()

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
0,Maria,McLaughlin,Philadelphia County,1901 John F Kennedy Blvd,Philadelphia,19103,1,Justice of the Supreme Court,1,COUNTYWIDE,...,False,101,0,1,10,1,COUNTYWIDE,COUNTY,NaN,130
1,Paula,Patrick,Philadelphia County,7017 Sherwood Rd,Philadelphia,19151,1,Justice of the Supreme Court,1,COUNTYWIDE,...,False,101,0,1,10,1,COUNTYWIDE,COUNTY,NaN,130
2,Kevin,Brobson,Dauphin County,3400 Hickory Hollow Rd,Harrisburg,17112,2,Justice of the Supreme Court,1,COUNTYWIDE,...,False,101,0,1,10,1,COUNTYWIDE,COUNTY,NaN,130
3,Patricia A.,McCullough,Allegheny County,2701 Bingham Drive,Pittsburgh,15241,3,Justice of the Supreme Court,1223,COUNTYWIDE,...,False,215,4,4,10,1,COUNTYWIDE,COUNTY,NaN,130
4,Jill,Beck,Allegheny County,5342 Pocusset Street,Pittsburgh,15217,1,Judge of the Superior Court,781,COUNTYWIDE,...,False,188,14,37,10,1,COUNTYWIDE,COUNTY,NaN,140


In [4]:
candidate_df["Office Title"].value_counts()

School Director                       504
Member of Council                     466
Judge of Election                     243
Inspector of Election                 181
Constable                             116
Mayor                                 116
Tax Collector                          96
Commissioner                           79
Magisterial District Judge             72
Judge of the Court of Common Pleas     67
Supervisor                             36
School Director At-Large               14
Member of County Council               11
Auditor                                 7
Judge of the Commonwealth Court         6
Member of Council At-Large              6
Judge of the Superior Court             4
Justice of the Supreme Court            4
Commissioner At-Large                   4
Sheriff                                 2
Controller                              2
Treasurer                               1
Name: Office Title, dtype: int64

In [5]:
candidate_df[candidate_df["Office Title"].isin(["School Director","School Director At-Large"])]

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
104,Carlos A.,Thomas,NaN,7152 Upland St.,Pittsburgh,15208,1,School Director,729,DISTRICT 1,...,False,188,13,4,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
105,Sylvia C.,Wilson,NaN,P.O.Box 4812,Pittsburgh,15206,2,School Director,714,DISTRICT 1,...,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
106,Grace,Higginbotham,NaN,7103 Kedron St.,Pittsburgh,15208,3,School Director,726,DISTRICT 1,...,False,188,13,1,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
107,Sylvia C.,Wilson,NaN,P.O.Box 4812,Pittsburgh,15206,1,School Director,714,DISTRICT 1,...,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
108,Lamont,"Frazier, Jr.",NaN,5161 Hillcrest St.,Pittsburgh,15224,1,School Director,691,DISTRICT 3,...,False,188,10,19,4,1,CITY SCHOOLS,PGHSCH3,NaN,710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,Todd,Kosenina,NaN,47 Trotter Ln.,Clinton,15026,2,School Director,204,WEST ALLEGHENY REGION 3,...,False,141,0,1,4,1,SCH/REGION,SCH0070,NaN,1000
690,Steven Russell,Pettit,NaN,420 Colt Cir,Clinton,15026,1,School Director,206,WEST ALLEGHENY REGION 3,...,False,141,0,3,4,1,SCH/REGION,SCH0070,NaN,1000
691,Todd,Kosenina,NaN,47 Trotter Ln.,Clinton,15026,2,School Director,204,WEST ALLEGHENY REGION 3,...,False,141,0,1,4,1,SCH/REGION,SCH0070,NaN,1000
692,Charles B.,Wade,NaN,"531 Park Ave,",Clariton,15025,1,School Director,126,CLAIRTON REGION 4,...,False,123,4,2,4,1,SCH/REGION,SCH0012,NaN,1010


# Taking a look at School Directors
## Data Cleaning

In [6]:
school_df = candidate_df[candidate_df["Office Title"].isin(["School Director","School Director At-Large"])]

In [7]:
#lets not bother w pittsburgh
school_df = school_df[school_df.City!="Pittsburgh"]

In [8]:
school_df["District"].value_counts()

MCKEESPORT AREA            24
MOON AREA                  22
GATEWAY                    21
WEST MIFFLIN AREA          14
RIVERVIEW                  12
DEER LAKES                 10
NORTH ALLEGHENY            10
CHARTIERS VALLEY           10
PINE RICHLAND REGION 2     10
BETHEL PARK                10
SOUTH ALLEGHENY            10
SOUTH FAYETTE               8
ELIZABETH FORWARD           8
ALLEGHENY VALLEY            8
HAMPTON                     8
QUAKER VALLEY REGION 1      8
WEST JEFFERSON HILLS        8
STO-ROX                     6
PENN HILLS                  6
STEEL VALLEY                6
AVONWORTH                   6
MONTOUR REGION 3            6
SOUTH PARK                  5
WEST ALLEGHENY REGION 2     5
CARLYNTON                   4
PINE RICHLAND REGION 1      4
SHALER AREA REGION 2        4
PINE RICHLAND REGION 3      4
WEST ALLEGHENY REGION 3     4
HIGHLANDS REGION 2          4
WOODLAND HILLS REGION 1     4
QUAKER VALLEY REGION 3      4
DUQUESNE                    3
STO-ROX RE

In [9]:
#????????? what
school_df[school_df["District"]=="COUNTYWIDE"]

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID


In [10]:
school_df = school_df[school_df["District"]!="COUNTYWIDE"]

In [11]:
# this is way too many columns
school_df.head()

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
191,Mary,Yuhas,NaN,158 W. Marigold St.,Munhall,15120,1,School Director At-Large,471,STEEL VALLEY,...,False,175,0,9,4,3,SCHOOL,SCH0063,NaN,890
192,William Robert,McLaughlin,NaN,3461 PinewoodDr.,West Homestead,15120,2,School Director At-Large,1244,STEEL VALLEY,...,False,221,0,3,4,3,SCHOOL,SCH0063,NaN,890
193,Cara Perhacs,Karstetter,NaN,3025 Alberta St.,Munhall,15120,3,School Director At-Large,468,STEEL VALLEY,...,False,175,0,6,4,3,SCHOOL,SCH0063,NaN,890
194,Mary,Yuhas,NaN,158 W. Marigold St.,Munhall,15120,1,School Director At-Large,471,STEEL VALLEY,...,False,175,0,9,4,3,SCHOOL,SCH0063,NaN,890
195,Cara Perhacs,Karstetter,NaN,3025 Alberta St.,Munhall,15120,2,School Director At-Large,468,STEEL VALLEY,...,False,175,0,6,4,3,SCHOOL,SCH0063,NaN,890


In [12]:
school_indices = list(school_df.index)

In [13]:
school_cleaned = pd.DataFrame()

In [14]:
name = []
party = []
district = []
for i in school_indices:
    row = school_df[school_df.index==i]
    name.append(row["First Name"].to_string(index=False) + " " + row["Last Name"].to_string(index=False))
    party.append(row["Party"].to_string(index=False))
    district.append(row["District"].to_string(index=False))

In [15]:
district2 = []
ward = []
for x in district:
    if "REGION" in x:
        parts = x.split("REGION")
        ward.append(int(parts[-1]))
        district2.append(' '.join(parts[:-1]).strip())
    else:
        ward.append(np.nan)
        district2.append(x)
    

In [16]:
district3 = []
for x in district2:
    if x == "PINE RICHLAND":
        district3.append("PINE-RICHLAND")
    else:
        district3.append(x)

In [17]:
school_cleaned["name"] = name
school_cleaned["party"] = party
school_cleaned["district"] = district3
school_cleaned["ward"] = ward

In [18]:
school_cleaned.head()

,name,party,district,ward
0,Mary Yuhas,Democratic,STEEL VALLEY,NaN
1,William Robert McLaughlin,Democratic,STEEL VALLEY,NaN
2,Cara Perhacs Karstetter,Democratic,STEEL VALLEY,NaN
3,Mary Yuhas,Republican,STEEL VALLEY,NaN
4,Cara Perhacs Karstetter,Republican,STEEL VALLEY,NaN


In [19]:
school_pos_df = pd.read_csv("schoolpos.csv")

In [20]:
school_pos_df.columns = ["num","district","term","ward"]

In [21]:
school_pos_df.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4-year,NaN
1,4,AVONWORTH,4-year,NaN
2,4,BALDWIN-WHITEHALL,4-year,NaN
3,4,BETHEL PARK,4-year,NaN
4,4,BRENTWOOD,4-year,NaN


In [22]:
school_pos_df.term = school_pos_df.term.map(lambda x: int(x.split("-")[0]))

In [23]:
school_pos_df.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN


In [24]:
school_pos_cleaned = school_pos_df[school_pos_df.ward.notna()==False]

In [25]:
school_pos_cleaned.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN


In [26]:
ward_df = school_pos_df[school_pos_df.ward.notna()==True]

In [27]:
row_list = []
for row in range(len(ward_df)):
    regions = ward_df.iloc[[row]].ward.values[0].split(",")
    if len(regions)>1:
        for x in regions:
            temp_dict = {
                "num": 1, 
                "district": ward_df.iloc[row,1],
                "term": ward_df.iloc[row,2],
                "ward": x
            }
            row_list.append(temp_dict)  
    else:
        temp_dict = {
            "num": ward_df.iloc[row,0], 
            "district": ward_df.iloc[row,1],
            "term": ward_df.iloc[row,2],
            "ward": ward_df.iloc[row,3]
            }
        row_list.append(temp_dict)  

In [28]:
school_pos_cleaned = school_pos_cleaned.append(row_list)

In [29]:
school_pos_cleaned.ward.value_counts()

1           15
3           14
2           12
7            2
9            2
4            2
at-large     2
5            1
Name: ward, dtype: int64

In [30]:
school_pos_cleaned

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN
...,...,...,...,...
45,2,WEST ALLEGHENY,4,2
46,1,WOODLAND HILLS,4,1
47,1,WOODLAND HILLS,4,2
48,2,WOODLAND HILLS,4,3


In [31]:
school_pos_cleaned = school_pos_cleaned[school_pos_cleaned.district != "PITTSBURGH"]

## Data Analysis

In [32]:
school_cleaned.head()

,name,party,district,ward
0,Mary Yuhas,Democratic,STEEL VALLEY,NaN
1,William Robert McLaughlin,Democratic,STEEL VALLEY,NaN
2,Cara Perhacs Karstetter,Democratic,STEEL VALLEY,NaN
3,Mary Yuhas,Republican,STEEL VALLEY,NaN
4,Cara Perhacs Karstetter,Republican,STEEL VALLEY,NaN


In [33]:
school_cleaned.district.value_counts()

MCKEESPORT AREA         24
MOON AREA               22
GATEWAY                 21
PINE-RICHLAND           18
WEST MIFFLIN AREA       14
QUAKER VALLEY           12
RIVERVIEW               12
STO-ROX                 10
WEST ALLEGHENY          10
BETHEL PARK             10
NORTH ALLEGHENY         10
SOUTH ALLEGHENY         10
DEER LAKES              10
CHARTIERS VALLEY        10
WEST JEFFERSON HILLS     8
ELIZABETH FORWARD        8
ALLEGHENY VALLEY         8
HAMPTON                  8
SOUTH FAYETTE            8
HIGHLANDS                8
STEEL VALLEY             7
CLAIRTON                 7
AVONWORTH                6
PENN HILLS               6
SHALER AREA              6
MONTOUR                  6
WOODLAND HILLS           5
SOUTH PARK               5
EAST ALLEGHENY           4
CARLYNTON                4
DUQUESNE                 3
PLUM                     2
CORNELL                  2
Name: district, dtype: int64

In [34]:
school_pos_cleaned.district.value_counts()

EAST ALLEGHENY          5
STO-ROX                 5
CLAIRTON                5
WOODLAND HILLS          4
FOX CHAPEL AREA         4
PINE-RICHLAND           3
HIGHLANDS               3
WEST ALLEGHENY          3
QUAKER VALLEY           3
KEYSTONE OAKS           3
SHALER AREA             3
STEEL VALLEY            2
BRENTWOOD               2
SOUTH ALLEGHENY         2
MONTOUR                 2
CHARTIERS VALLEY        2
NORTH HILLS             1
PENN HILLS              1
ELIZABETH-FORWARD       1
DEER LAKES              1
HAMPTON                 1
WEST JEFFERSON HILLS    1
SOUTH FAYETTE           1
ALLEGHENY VALLEY        1
BETHEL PARK             1
NORTH ALLEGHENY         1
NORTHGATE               1
WEST MIFFLIN AREA       1
WILKINSBURG             1
RIVERVIEW               1
MOUNT LEBANON           1
MCKEESPORT AREA         1
MOON AREA               1
GATEWAY                 1
BALDWIN-WHITEHALL       1
CORNELL                 1
DUQUESNE                1
PLUM                    1
CARLYNTON   

In [35]:
no_candidates = []
for dis in school_pos_cleaned.district:
    if dis not in list(school_cleaned.district):
        no_candidates.append(dis)

In [37]:
no_candidates = list(set(no_candidates))
no_candidates = [x.capitalize() for x in no_candidates]
for x in sorted(no_candidates):
    print(x)

Baldwin-whitehall
Brentwood
Elizabeth-forward
Fox chapel area
Keystone oaks
Mount lebanon
North hills
Northgate
Upper st. clair
Wilkinsburg
